In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import random
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import mean_squared_error, confusion_matrix, r2_score
from sklearn.inspection import permutation_importance
import shap
from sklearn.linear_model import LinearRegression, LogisticRegression



df = pd.read_csv('MH_encoded.csv')
df.head()

,Age,Years_of_Experience,Hours_Worked_Per_Week,Number_of_Virtual_Meetings,Work_Life_Balance_Rating,Stress_Level,Access_to_Mental_Health_Resources,Productivity_Change,Social_Isolation_Rating,Satisfaction_with_Remote_Work,...,Work_Location_Onsite,Work_Location_Remote,Mental_Health_Condition_Burnout,Mental_Health_Condition_Depression,Mental_Health_Condition_None,Region_Asia,Region_Europe,Region_North America,Region_Oceania,Region_South America
0,32,13,47,7,2,1.0,0,0.0,1,0.0,...,False,False,False,True,False,False,True,False,False,False
1,40,3,52,4,1,1.0,0,2.0,3,2.0,...,False,True,False,False,False,True,False,False,False,False
2,59,22,46,11,5,1.0,0,1.0,4,0.0,...,False,False,False,False,False,False,False,True,False,False
3,27,20,32,8,4,2.0,1,2.0,3,0.0,...,True,False,False,True,False,False,True,False,False,False
4,49,32,35,12,2,2.0,1,0.0,3,0.0,...,True,False,False,False,True,False,False,True,False,False


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4629 entries, 0 to 4628
Data columns (total 38 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Age                                 4629 non-null   int64  
 1   Years_of_Experience                 4629 non-null   int64  
 2   Hours_Worked_Per_Week               4629 non-null   int64  
 3   Number_of_Virtual_Meetings          4629 non-null   int64  
 4   Work_Life_Balance_Rating            4629 non-null   int64  
 5   Stress_Level                        4629 non-null   float64
 6   Access_to_Mental_Health_Resources   4629 non-null   int64  
 7   Productivity_Change                 4629 non-null   float64
 8   Social_Isolation_Rating             4629 non-null   int64  
 9   Satisfaction_with_Remote_Work       4629 non-null   float64
 10  Company_Support_for_Remote_Work     4629 non-null   int64  
 11  Physical_Activity                   4629 no

In [4]:
from sklearn.preprocessing import StandardScaler

# Define the numeric features to be standardized
numeric_features = [
    'Age',
    'Years_of_Experience',
    'Hours_Worked_Per_Week',
    'Number_of_Virtual_Meetings',
    'Work_Life_Balance_Rating',
    'Stress_Level',
    'Social_Isolation_Rating',
    'Company_Support_for_Remote_Work'
]

# Initialize the scaler
scaler = StandardScaler()

# Fit and transform the numeric features
df[numeric_features] = scaler.fit_transform(df[numeric_features])

In [5]:
x = df.drop(['Work_Life_Balance_Rating'], axis=1)
y = df['Work_Life_Balance_Rating']

x_train, x_val, y_train, y_val = train_test_split(
    x, y, test_size=0.8, random_state=23)

In [6]:
y_train = y_train.fillna(0)

numerical_features = x.columns

numeric_transformer = StandardScaler()

In [8]:
preProcess = ColumnTransformer(
    transformers=[('num', numeric_transformer, numerical_features)])

pipeline = Pipeline([('preprocess', preProcess),
                     ('classifier', RandomForestClassifier())])

param_grid = [{'classifier': [RandomForestClassifier()],
               'regressor__max_depth': [1, 2, 3, 4],
              'regressor__n_estimators': [100, 500]}]

In [9]:
random.seed(1)
cv_mine = KFold(n_splits=3)
gridSearch = GridSearchCV(pipeline, param_grid, cv=cv_mine,
                          scoring='r2', n_jobs=-1)
gridSearch.fit(x_train, y_train)


KeyboardInterrupt

